# Creating Database from Web Scraping Social Networking Website and SQLite

Source Data: https://en.wikipedia.org/wiki/List_of_social_networking_websites

# Imports packages and data retrieval steps

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import requests# used for loading webpages for source code
import urllib.request # used to open URLs 
import pandas as pd # to manipulated data
import sqlite3 #deploy SQL database
import lxml.html # Convert HTML text
from bs4 import BeautifulSoup # HTML/XML parser for Python
#pulling data out of HTML and XML
# Beautiful Soup Documentation
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
import math

Request URL

In [2]:
url    = "https://en.wikipedia.org/wiki/List_of_social_networking_websites"
#url    = "https://www.dreamgrow.com/top-15-most-popular-social-networking-sites/"
source = urllib.request.urlopen(url)
soup   = BeautifulSoup(source, 'lxml')
title  = soup.title.text
print(title)


List of social networking websites - Wikipedia


Locate table data of interest

In [14]:
#table info
info_table      = soup.find('table', class_='wikitable sortable')
info_table_rows = info_table.find_all('tr')
all_tables      = soup.findAll('table')
response        = requests.get(url)
headers         = (["Name", "Description_focus", "Date_launched",
              "Registered_users", "Registration", "Global_Alexa_page_ranking"])
#print(all_tables)

Change Registered user to a string data type

In [15]:
Registered_users = []
str = ""
for Registered_user in Registered_users:
    info= Registered_users.text
    str+=info

In [5]:
df.Registered_users.value_counts()

NameError: name 'df' is not defined

Parse table into a DataFrame


In [ ]:
df         = pd.read_html(response.content, skiprows = 1)[0]
df.columns = headers
type(df) #pandas.core.frame.DataFrame
print("Total number of SNW: ", len(df),'\n')
print("First 5 rows from the data table:", df.head(5),'\n')
print("Table Information:")
print(df.info(),'\n')


# Created SQL table

In [ ]:
#Make database and will only exist in memory
db= sqlite3.connect(':memory:')#use the argument ":memory:" to create a temporary database in the RAM
cursor = db.cursor()# used to execute the social media query being made
cursor.execute("""
  CREATE TABLE SOCIAL_MEDIA_WEBSITES("Name", "Description_focus",
                                     "Date_launched", "Registered_users",
                                     "Registration", "Global_Alexa_page_ranking") 
""") # Create Database table

#Inserting Data into the Database
for row in df.itertuples():
   insert_sql_syntax = """
   INSERT INTO SOCIAL_MEDIA_WEBSITES("Name", "Description_focus",
                                     "Date_launched", "Registered_users",
                                     "Registration", "Global_Alexa_page_ranking")
              VALUES (?,?,?,?,?,?)
              """
   cursor.execute(insert_sql_syntax, row[1:])
db.commit()
## extract info from db
for row in cursor.execute("""
  SELECT Description_focus
  FROM SOCIAL_MEDIA_WEBSITES
  WHERE Name = "Instagram"
"""):
  print('This is the 1st output of my sqlite Database:\n==================================================')
print('Instagram is', row)  

print('\n==================================================')

for row in cursor.execute("""
    SELECT Registered_users
    FROM SOCIAL_MEDIA_WEBSITES
    WHERE Name = "Instagram"
"""):
    print('This is the 2nd output of my sqlite Database:\n==================================================')
print('Instagram registered users total', row)


# Pulling in the data from my database to create graph



In [ ]:
# Looking at first 5 rows of the dataset
df.head()

In [ ]:
print("There are {} observations and {} features in this dataset. \n".format(df.shape[0],df.shape[1]))

print("There are {} types of social networking sites in this dataset such as {}... \n".format(len(df.Description_focus.unique()),
                                                                           ", ".join(df.Description_focus.unique()[0:5])))

print("There are {} registered users on social networking sites in this dataset such as {}... \n".format(len(df.Registered_users.unique()),
                                                                                                             ", ".join(df.Registered_users.unique()[0:5])))


In [ ]:
def graph_data():
    cursor.execute("""
  SELECT Name, 
  Registered_users, 
  Global_Alexa_page_ranking 
  FROM SOCIAL_MEDIA_WEBSITES 
""")
    

    Name                        = []
    Registered_users            = []
    Global_Alexa_page_ranking   = []


    for row in cursor.fetchall():
        #print(row[0])
        Name.append(row[0])
        Registered_users.append(int(row[1].replace(',', '').strip(), base=16))
        Global_Alexa_page_ranking.append(str(row[2].replace(',','').strip(), base=16))

# calling graph function 
graph_data()

In [ ]:
# Pulling in the data from my database
cursor.execute("""
  SELECT Name, Registered_users, Global_Alexa_page_ranking FROM SOCIAL_MEDIA_WEBSITES 
""")

Name                        = []
Registered_users            = []
Global_Alexa_page_ranking   = []


for row in cursor.fetchall():
    Name.append(row[0])
    Registered_users.append(int(row[1].replace(',', '').strip()))
    Global_Alexa_page_ranking.append(int(row[2].replace(',','').strip()))
    

# Creating a dataframe out of all the rows from database
plot1 = {
    'Name':    [],
    'Registered_users':   [],
    'Global_Alexa_page_ranking':    []   
}
for row in Name:
    plot1['Name'].append(row)
for row in Users:
    plot1['Registered_users'].append(row)
for row in Rank:
    plot1['Global_Alexa_page_ranking'].append(row)
df1 = pd.DataFrame(plot1)

df1.iloc[180,0] = 'Social Users'


In [ ]:
cursor.execute("""
    SELECT Name, Registered_users, Global_Alexa_page_ranking FROM SOCIAL_MEDIA_WEBSITES 
""")

Name                        = []
Registered_users            = []
Global_Alexa_page_ranking   = []


for row in cursor.fetchall():
    Name.append(row[0])
    Registered_users.append(int(row[1].replace(',', '').strip()))
    Global_Alexa_page_ranking.append(int(row[2].replace(',','').strip()))
print(row,'\n')   

In [ ]:

# Pulling in a seperate file for Monthly Active User merging the dataframes to plot
rr = requests.get('https://www.dreamgrow.com/top-15-most-popular-social-networking-sites/')

htmltext = lh.fromstring(rr.text)

#table info
gdp = htmltext.get_element('table') 

gdpbody = gdp[1].getchildren()

plot2 = {
    'Name' : [],
    'Monthly Active Users'     : []
}
for i in gdpbody:
        plot2['Name'].append(i.getchildren()[0].text_content())
        plot2['Monthly Active Users'].append(float(i.getchildren()[1].text_content().replace(',','').strip()))

df2=pd.DataFrame(plot2)

df3 = pd.merge(df1, df2, left_on = 'Social Network', right_on = 'Social Network')

df3['Inactive_Users'] = (df3['Monthly Active Users']*1000000000) /df3['Users']
#Mutiplied GDP by 1B since it is represented in billions

df4 = df3.copy()

# Creating sizes and texts  for my plot
hover_text = []
bubble_size = []
bubble_ref =[]

for index, row in df3.iterrows():
    hover_text.append(('Social Network: {Social Network}<br>'+
                      'Monthly Active Users : {Monthly Active Users}<br>'+
                      'Users : {Users}<br>'+
                      'Inactive_Users : {Inactive_Users}<br>').format(Social_Network=row['Social Network'],
                                                             Users = row['Users'],
                                                             Monthly_Active_User = row['Monthly Active User'],
                                                             UserPer = row['Inactive_Users']
                                                                                                ))
    bubble_size.append(row['Users'])
    bubble_ref.append(math.sqrt(row['Users']))


df3['text']= hover_text
df3['size'] = bubble_size
df3['ref']=bubble_ref
sizeref = (max(df3['ref']))/350

#close database
db.close()